#  Viterbri Algorithm


Viterbi Algorithm is usually used for finding the most likely tag sequence under an HMM. 


In [2]:
import re

In [3]:
def viterbi_alg(tokens, states, trans_probs, emis_probs):
    
    viterbi = [[0]*len(states) for i in range(len(tokens))]
    viterbi[0][0] = 1.0
    way=[[None]*len(states) for i in range(len(tokens))]
    for i in range(1, len(tokens)):
        word = tokens[i]
        for j in range(1, len(states)):
            pos = states[j]
            if i == 1:
                ppos = states[0]
                t = trans_probs[(ppos, pos)]
                e = emis_probs[(pos, word)]
                viterbi[i][j]= e*t
                way[i][j] = (0, 0)
            else:
                e = emis_probs[(pos, word)]
                m = 0
                for k in range(1, len(states)-1):
                    x = viterbi[i-1][k]
                    ppos = states[k]
                    t = trans_probs[(ppos, pos)]
                    #viterbi[i][j] = max(viterbi[i][j], e*t*x)  
                    if viterbi[i][j] < e*t*x:
                        viterbi[i][j] = e*t*x
                        way[i][j] = (i-1, k)
    return viterbi, way

Transition Probability is probability of going from one STATE to the other, e.g. from one word to the other (modeling word sequences)

Emission Probability = probability of 'emitting' a PARTICULAR WORD when in a particular state 
the main weakness of HMM's is tagging unknown words

In [4]:
# transition probabilities
trans_probs = {('[START]', 'verb'): 0.2,
			   ('[START]', 'noun'): 0.8,
			   ('noun', 'verb'): 0.8,
			   ('noun', '[END]'): 0.1,
			   ('noun', 'noun'): 0.1,
			   ('verb', 'noun'): 0.2,
			   ('verb', 'verb'): 0.1,
			   ('verb', '[END]'): 0.7
			   }

# emission probabilities			   
emis_probs = {('noun', 'fish'): 0.8,
			  ('noun', 'sleep'): 0.2,
			  ('verb', 'fish'): 0.5,
			  ('verb', 'sleep'): 0.5,
              ('[START]', '#') : 1.0,
              ('[END]', '$') : 1.0
			  }

states = ['[START]', 'verb', 'noun', '[END]']
s = 'fish sleep'
tokens = ['#']
tokens.extend(re.findall('\w+', s.lower()))
tokens.append('$')

In [5]:
for i in states:
    for j in states:
        if (i, j) not in trans_probs:
            trans_probs[(i, j)] = 0.0

for i in states:
    for j in tokens:
        if (i, j) not in emis_probs:
            emis_probs[(i, j)] = 0.0

In [6]:
viterbi, way = viterbi_alg(tokens, states, trans_probs, emis_probs)

In [7]:
#восстановление пути
i = len(tokens) - 1
j = len(states) - 1
p = [(i, j)]
#print(viterbi, '\n\n', way)
while i != 0 and j != 0:
    pair = way[i][j]
    i = pair[0]
    j = pair[1]
    p.append(pair)
#rint(p[::-1])

for i,j in p[::-1]:
    print(tokens[i], states[j], viterbi[i][j])

# [START] 1.0
fish noun 0.6400000000000001
sleep verb 0.25600000000000006
$ [END] 0.17920000000000003
